References

https://www.pyimagesearch.com/2014/08/04/opencv-python-color-detection/ 

https://www.geeksforgeeks.org/detection-specific-colorblue-using-opencv-python/ (basic colour mask)

https://www.learnopencv.com/color-spaces-in-opencv-cpp-python/ (colour space)

https://www.sparkfun.com/news/2191?__hstc=77938635.35c34ab7bf88e972fdd7a7debc8575ba.1474848000139.1474848000140.1474848000141.1&__hssc=77938635.1.1474848000142&__hsfp=1773666937 

https://docs.opencv.org/2.4/modules/imgproc/doc/histograms.html?highlight=comparehist#comparehist (SetHistBinRanges, ThreshHist, ReleaseHist, ClearHist)

https://docs.opencv.org/2.4/doc/tutorials/imgproc/histograms/histogram_comparison/histogram_comparison.html 

https://stackoverflow.com/questions/32414559/opencv-contour-minimum-dimension-location-in-python 

https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contour_features/py_contour_features.html (contour features)

https://docs.opencv.org/2.4/doc/tutorials/imgproc/histograms/histogram_calculation/histogram_calculation.html#code 

https://github.com/abidrahmank/OpenCV2-Python/blob/master/Official_Tutorial_Python_Codes/3_imgproc/comparehist.py (custom histogram comparisons)


In [2]:
# Adapted from pycharm IDE - running python 3.6 - some expected conversion errors

import numpy as np
import cv2
import time
#import ffmpeg


If needed, convert MP4 to avi directly first, might be best to run directly from command window

In [ ]:
ffmpeg_file path = \\Users\\User\\ffmpeg-4.1-win64-static\\bin\\ffmpeg.exe ## example path to ffmpeg if path variable is not set or working
Original = M:\Data\GoPro_Visor\GoPro_Video\003_setup.MP4 # example mp4
Output = M:\Data\GoPro_Visor\Converted_Video\003_setup.avi # example output .avi format

ffmpeg_file path -i Original Output

Define video manipulation functions

In [3]:
# Histogram equalization parameters

def split_into_rgb_channels(image):
  '''Split the target image into its red, green and blue channels.
  image - a numpy array of shape (rows, columns, 3).
  output - three numpy arrays of shape (rows, columns) and dtype same as
           image, containing the corresponding channels. 
  '''
  red = image[:,:,2]
  green = image[:,:,1]
  blue = image[:,:,0]
  return red, green, blue

def equalizeHistColor(frame):
    # equalize the histogram of color image
    img = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)  # convert to HSV
    img[:, :, 2] = cv2.equalizeHist(img[:, :, 2])  # equalize the histogram of the V channel
    return cv2.cvtColor(img, cv2.COLOR_HSV2RGB)  # convert the HSV image back to RGB format


In [4]:
# What file are we inputting?
# Specify file pathway unless in the GoPro_Grid_Pi directory
webcam = 0 # set to 1 if input is a webcam

part = '003' # Version - example '001' or '054'
exp = '_camera_p3' # ex. '003_camera_p3'
in_format = '.avi'
in_file = part + exp + in_format

if webcam == 1:
    in_file = ''

In [ ]:
# Are we saving an output file (file with overlaid filters/bounders/manipulations)?

 # # Version - example '001' or '054'
out_format = '.avi'
out_file = part + exp + out_format

# Output file parameter
imgSize=(640,480) # likely best to set to original  dimensions
frame_per_second=30.0
writer = cv2.VideoWriter(out_format, cv2.VideoWriter_fourcc(*"MJPG"), frame_per_second,imgSize,False)

Initialize event times of the appropriate participant

In [5]:
#for 003 (video 0079)%%%
start_eeg = 652
door_closed = 4800
start_flash = 8897

In [ ]:


#for 004 (video 0080)%%%
start_eeg = 3041
door_closed = 6947
start_flash = 12159

#for 005 (video 0081)%%%
start_eeg = 3330
door_closed = 12240
start_flash = 3268+14400

#for 006 (video 0082?)%%%
start_eeg = 567
door_closed = 5040
start_flash = 10446

#for 007 (video 0083?)%%%
start_eeg = 1045
door_closed = 7440
start_flash = 12567
#off_flash =

#for 008 (video 0084?)%%%
start_eeg = 2053
door_closed = 7680
start_flash = 12673

#for 009 (video 0084?)%%%
start_eeg = 616
door_closed = 6000
start_flash = 11040

#for 010 (video 0087)%%%
start_eeg = 1443
door_closed = 8640
start_flash = 13343

#for 011 (video 0088)%%%
start_eeg = 638
door_closed = 8400
start_flash = 13176

Initalize doubles for start/end of flash trains + state list for each frame

In [6]:
# First Pass
# List of [frame + start event trigger] (where the max[index] = corresponds with the last EEG event)
Trigger_Start = []
# List of [frame + end event trigger]
Trigger_Stop = []
# List of [frame + trigger state] (0 B + G channels below thresholds, 1 above B channel threshold, 2 above R channel threshold
Tigger_State = []

# Want a frame of each start trigger saved to a folder

# Second Pass for extracting epochs based off first pass - figure out later
# Eventually will output an ~[-1,1] video epoch to be the raw input for deep learning
Trigger_Epoch = []

In [7]:
## frame dimensions (x,y) & img dimensions (2x,2y) # for resizing drawn frames
scaling_factorx=1 
scaling_factory=1 
scaling_factor2x=1 
scaling_factor2y=1

# Manipulation Variables,
kernelSize = 21
GB_Kernel = 21

# Edge Detection Parameter
parameter1=20
parameter2=60

intApertureSize=1
# Colours
custom_color_list = []
custom_color_list = \
    ["COLOR_BGR2RGB",
     "COLOR_RGB2BGR",
     "COLOR_BGR2GRAY",
     "COLOR_RGB2GRAY",
     "COLOR_BGR2HSV",
     "COLOR_RGB2HSV",
     "COLOR_RGB2HLS",
     "COLOR_BGR2HLS",
     "COLOR_BGR2XYZ",
     "COLOR_RGB2XYZ",
     "COLOR_BGR2Lab",
     "COLOR_RGB2Luv"]

custom_color_type = 1 # 1-12 - look at elements in custom_color_list

# Thresholding 
threshold1=100
threshold2=200

# Contours
color=(255,0,0)
thickness=2
count = 0

Run through each frame from door closed indexed from start_eeg


In [18]:
# Find a given colours hsv value to threshold around

green = np.uint8([[[0,255,0 ]]])
hsv_green = cv2.cvtColor(green,cv2.COLOR_BGR2HSV)
print('green hsv = {}'.format(hsv_green))
#[[[ 60 255 255]]]

# red = np.uint8([[[255,0,0 ]]])
# hsv_red = cv2.cvtColor(red,cv2.COLOR_BGR2HSV)
# print('red hsv = ' + hsv_red)

# blue = np.uint8([[[255,0,0 ]]])
# hsv_blue = cv2.cvtColor(blue,cv2.COLOR_BGR2HSV)
# print('blue hsv = ' + hsv_blue)
      


green hsv = [[[ 60 255 255]]]


Define the colour boundaries + size of detector + buffer of frames around event if outputting epochs

In [ ]:


# define the lower and upper boundaries of the "green" ball in
# the HSV color space
# redLower = ()
# redUpper = ()
greenLower = (29, 86, 6)
greenUpper = (64, 255, 255)
blueLower = (86, 31, 4)
blueUpper = (220, 88, 50)
colours = [red, green, blue]
for i in len(colours)
# bgr = [40, 158, 16]
# thresh = 20
# minBGR = np.array([bgr[0] - thresh, bgr[1] - thresh, bgr[2] - thresh])
# maxBGR = np.array([bgr[0] + thresh, bgr[1] + thresh, bgr[2] + thresh])  
# maskBGR = cv2.inRange(bright,minBGR,maxBGR)
# resultBGR = cv2.bitwise_and(bright, bright, mask = maskBGR)



In [20]:
#get the frame number
#start = cam.get(cv2.CAP_PROP_POS_FRAMES)
#print(start)

NameError: name 'cam' is not defined

In [ ]:
cap = cv2.VideoCapture(in_file)  # load the video
cap.set(2,frame_no) # first arguement of 2 indicated a zero-based index - and second indicates the frame_number that is the focus
counter = 0 # which frame
while (cap.isOpened()):  # play the video by reading frame by frame
    ret, frame = cap.read()
    if ret == True:
        split_into_rgb_channels(frame) # seperate into 3 seperate RGB frames
        
        # equalize the histogram of color image - contrast enhancement
        frame1 = equalizeHistColor(frame)

        # Smoothing
        gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (GB_Kernel, GB_Kernel), 0)
        # convert to HSV
        img_filter = cv2.cvtColor(img_filter, cv2.COLOR_RGB2HSV)
        
        mask = cv2.inRange(hsv, greenLower, greenUpper)

        #img_binary_red = cv2.inRange(img_filter.copy(), THRESHOLD_LOW, THRESHOLD_HIGH)
        img_binary_green = cv2.inRange(img_filter.copy(), greenLower, greenUpper)
        img_binary_blue = cv2.inRange(img_filter.copy(), blueLower, blueUpper)
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)
        
        # The bitwise and of the frame and mask is done so that only the blue coloured objects are highlighted and stored in res 
        res = cv2.bitwise_and(frame,frame, mask= mask) 
        cv2.imshow('frame',frame) 
        cv2.imshow('mask',mask) 
        cv2.imshow('res',res) 
        
        img_contours = img_binary.copy()
        contours = cv2.findContours(img_contours, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
        # Thresholding
        # ret, mask = cv2.threshold(blur, threshold1, threshold2, cv2.THRESH_BINARY)
        # ret, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        # ret, mask = cv2.threshold(blur,threshold1, threshold2,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#         mask = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
#         mask = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
#         kernel = np.ones((3, 3), np.uint8)
#         mask = cv2.erode(mask, kernel, iterations=7)  # morphology erosion
#         mask = cv2.dilate(mask, kernel, iterations=5)  # morphology dilation

#         mask_inv = cv2.bitwise_not(mask)
#         img = cv2.bitwise_and(frame1, frame1, mask=mask_inv)
#         img = cv2.addWeighted(frame1, 0.1, img, 0.9, 0)

        #Contouring - includes a bounding box - takes from previous gray box
        ret, thresh = cv2.threshold(gray, 75, 100, cv2.THRESH_BINARY_INV)   #cv2.CHAIN_APPROX_SIMPLE
        img1, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        if len(contours) != 0:
            c = max(contours, key=cv2.contourArea)  # find the largest contour
            area = cv2.contourArea(c) # area of the contour
            x, y, w, h = cv2.boundingRect(c)  # get bounding box of largest contour
            img2=cv2.drawContours(img, c, -1, color, thickness) # draw largest contour
            # img2 = cv2.drawContours(frame, contours, -1, color, thickness)  # draw all contours
            img3 = cv2.rectangle(img2, (x, y), (x + w, y + h), (0, 0, 255), 2)  # draw red bounding box in img
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img3, "label", (x, h), font, 1,(255,255,255),2)
            
#             # Print out the location and size (radius) of the largest detected contour
#         if center != None:
#             print str(center) + " " + str(radius) 

        from this point on working soely with largest contour   
        # will output the largest contours (RGB value of each contour mask)
        if count <10
            baseline = count 
        elif baseline = average over the last 10 frame
            
        old_baseline = baseline
        average hist
        # comparing two hists
        double base_base = compareHist( hist_base, hist_base, compare_method )
        
        if diff = 1 # if different then don't update baseline - otherwise it will washout the difference
            
        ###########################################################
        # Find the largest contour and use it to compute the min enclosing circle
        center = None
        radius = 0
        if len(contours) > 0:
            c = max(contours, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            if M["m00"] > 0:
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                if radius < MIN_RADIUS:
                    center = None
        
#         for i, cnt in enumerate(contours):

#            # compute the bounding box for the contour
#            (x, y, w, h) = cv2.boundingRect(cnt)

#            # reject contours outside size range
#            if w > 250 or w < 30 or h > 250 or h < 30 :
#                   continue

#            # make sure the box is inside the frame
#            if x <= 0 or y <= 0 or x+w >= (WIDTH -1) or y+h >= (HIGHT -1):
#                   continue


        
        #######################################################
        colors = ('b', 'g', 'r')

        for i, col in enumerate(colors):
            hist[i] = cv2.calcHist([imgDEF], [i], None, [256], [1, 256])
            plt.plot(hist, color=col)

        plt.show()
        
        # Getting RGB channel sepecifc histograms - this is C+
        #####################################################
        // Quantize the hue to 30 levels
        // and the saturation to 32 levels
        int hbins = 30, sbins = 32;
        int histSize[] = {hbins, sbins};
        // hue varies from 0 to 179, see cvtColor
        float hranges[] = { 0, 180 };
        // saturation varies from 0 (black-gray-white) to
        // 255 (pure spectrum color)
        float sranges[] = { 0, 256 };
        const float* ranges[] = { hranges, sranges };
        MatND hist;
        // we compute the histogram from the 0-th and 1-st channels
        int channels[] = {0, 1};
        
        ######################################################
        imgDEF = cv2.imread("YOUR_IMAGE.jpg")
        imgGray = cv2.cvtColor(imgDEF, cv2.COLOR_BGR2GRAY)    
        ghist = cv2.calcHist([imgGray], [0], None, [256], [0,256])

        print ("Mean = {:.1f}, standard deviation = {:.1f}, total = {:.0f}".format(
            np.mean(ghist).item(),
            np.std(ghist).item(),
            np.sum(ghist).item()
        ))

        plt.plot(ghist, color='m')
        plt.show()
        #########################################################
        
        
        # Add frame # to the appropriate structures - Trigger_Start Trigger_Stop are either 1 (green) or 2 (blue), Trigger state_state can also be 0 (neither green nor blue)
        # List of [frame + start event trigger] (where the max[index] = corresponds with the last EEG event)
        if Trigger_Start = []
        # List of [frame + end event trigger]
        if Trigger_Stop = []
        # List of [frame + trigger state] (0 B + G channels below thresholds, 1 above B channel threshold, 2 above R channel threshold
        Tigger_State[count] = 
        else    
            
        writer.write(img)  # save the frame into video file
        count += 1
        if count % 1 == 0:
            # cv2.imshow('Original', frame)  # show the original frame
            cv2.imshow('New', img) #show the new frame
        if cv2.waitKey(1)& 0xFF == ord('q'):
            break
    else:
        print("ref != True")
        break
    # When everything done, release the capture
writer.release()
cap.release()
cv2.destroyAllWindows()